In [44]:
import pathlib
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pickle


In [45]:
main_dir=pathlib.Path(".")

In [79]:
data={
    "mutation":[],
    "ko":[],
    "reward":[],
    "agent":[]
}
sample_num=100
fig=go.Figure()
for ko in main_dir.iterdir():
    if ko.name.startswith(".") or ko.name.startswith("Data_Analysis.ipynb"):
        continue
    else: 
        agent1_ko,agent2_ko=ko.name.split("_")
        if (ko/"returns_1800.json").exists():
            with open(ko/"returns_1800.json") as f:
                rewards=json.load(f)
            data["reward"].extend(rewards["agent1"][-sample_num:])
            data["reward"].extend(rewards["agent2"][-sample_num:])
            data["mutation"].extend([agent1_ko]*sample_num+[agent2_ko]*sample_num)
            data["ko"].extend([ko.name]*sample_num*2)
            data["agent"].extend(["agent1"]*sample_num+["agent2"]*sample_num)
        
data=pd.DataFrame(data)    
        

In [80]:
fig.add_trace(go.Box(
    x=data[data["agent"]=="agent1"]["ko"],
    y=data[data["agent"]=="agent1"]["reward"]
))
fig.add_trace(go.Box(
    x=data[data["agent"]=="agent2"]["ko"],
    y=data[data["agent"]=="agent2"]["reward"]
))
fig.update_layout(boxmode="group",width=800)


In [81]:
data.drop(["agent"],axis=1,inplace=True)

In [3]:
import cobra

In [4]:
model=cobra.io.read_sbml_model("../../iAF1260.xml")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-03


In [2]:
exchange_reactions = {
    "serA": "EX_ser__L_e",
    "glyA": "EX_gly_e",
    "cysE": "EX_cys__L_e",
    "metA": "EX_met__L_e",
    "thrC": "EX_thr__L_e",
    "ilvA": "EX_ile__L_e",
    "trpC": "EX_trp__L_e",
    "pheA": "EX_phe__L_e",
    "tyrA": "EX_tyr__L_e",
    "hisB": "EX_his__L_e",
    "proA": "EX_pro__L_e",
    "argA": "EX_arg__L_e",
    "leuB": "EX_leu__L_e",
}

In [83]:
growth_values={}
for exchange in exchange_reactions:
    temp_model=model.copy()
    temp_model.reactions.get_by_id(exchange_reactions[exchange]).lower_bound=0.5
    temp_model.reactions.get_by_id(exchange_reactions[exchange]).upper_bound=0.5+1e-6
    growth_values[exchange]=temp_model.optimize().objective_value
    print(exchange,growth_values[exchange])


Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmp1p3jbv8u.lp
Reading time = 0.13 seconds
: 1668 rows, 4764 columns, 18462 nonzeros
serA 0.7156362913563521
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpgwa9js_u.lp
Reading time = 0.03 seconds
: 1668 rows, 4764 columns, 18462 nonzeros
glyA None
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpbadms1vy.lp
Reading time = 0.01 seconds
: 1668 rows, 4764 columns, 18462 nonzeros
cysE 0.6900577908102841
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpjty5k_yx.lp
Reading time = 0.01 seconds
: 1668 rows, 4764 columns, 18462 nonzeros
metA None
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpc1jvtesa.lp
Reading time = 0.01 seconds
: 1668 rows, 4764 columns, 18462 nonzeros
thrC 0.7000885753381546
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmp

In [84]:
growth_values

{'serA': 0.7156362913563521,
 'glyA': None,
 'cysE': 0.6900577908102841,
 'metA': None,
 'thrC': 0.7000885753381546,
 'ilvA': 0.6735069963393,
 'trpC': 0.6348220502221712,
 'pheA': 0.6514392703779905,
 'tyrA': 0.6549063461194169,
 'hisB': 0.6832870112539766,
 'proA': 0.6910608692630685,
 'argA': 0.6825347024143829,
 'leuB': 0.6800270062824142}

In [89]:
mean_df=data.groupby(["ko","mutation"]).median()
mean_df["std"]=data.groupby(["ko","mutation"]).std()
mean_df.reset_index(inplace=True)

In [91]:
mean_df["growth"]=mean_df.apply(lambda x:growth_values[x["mutation"]],axis=1)

In [92]:
mean_df

,ko,mutation,reward,std,growth
0,argA_cysE,argA,25.752612,42.862198,0.682535
1,argA_cysE,cysE,50.364028,42.668546,0.690058
2,argA_thrC,argA,40.660517,37.375820,0.682535
3,argA_thrC,thrC,43.167314,38.126717,0.700089
4,cysE_leuB,cysE,71.052767,57.166854,0.690058
5,cysE_leuB,leuB,27.508298,36.633873,0.680027
6,cysE_trpC,cysE,20.669014,43.660020,0.690058
7,cysE_trpC,trpC,113.706123,78.962289,0.634822
8,hisB_cysE,cysE,20.670265,63.208590,0.690058
9,hisB_cysE,hisB,39.966385,59.924141,0.683287


In [77]:
mean_df.groupby("ko")[["reward","growth"]].apply(lambda x:x.iloc[1]-x.iloc[0])

,reward,growth
ko,,
argA_cysE,22.039574,0.001505
argA_thrC,5.355440,0.003511
cysE_leuB,-41.917500,-0.002006
cysE_trpC,61.543022,-0.011034
hisB_cysE,14.943096,-0.001354
hisB_thrC,-42.291885,0.003360
ilvA_thrC,110.533221,0.005316
leuB_trpC,74.723531,-0.009028
metA_argA,-2.266346,NaN


In [139]:
import pickle
import torch

In [99]:
with open("argA_thrC/final_batch_obs.pkl","rb") as f:
    obs=pickle.load(f)

In [168]:
torch.cat([tens[:,:,None] for tens in obs["agent1"][:,[0,1]].split(40,dim=0)],dim=2)[-1,:,4]

tensor([13.8850, 45.2514])